In [1]:
import pandas as pd
import clickhouse_driver
from typing import Tuple

import warnings
warnings.filterwarnings('ignore')

In [2]:
CH_HOST = 'localhost'
CH_PORT = int('9000')
CH_DATABASE = 'nir24'
CH_USERNAME = 'user'
CH_PASSWORD = 'password'

In [3]:
all_raw_logs = pd.read_csv('./logon+device+file_order_by_datetime.csv')
#all_logs['day'] = pd.to_datetime(all_logs['date'])
all_raw_logs

,id,datetime,user,pc,type,metadata
0,{X1D9-S0ES98JV-5357PWMI},2010-01-02 06:49:00,NGF0157,PC-6056,logon,"{""activity"": ""Logon""}"
1,{G2B3-L6EJ61GT-2222RKSO},2010-01-02 06:50:00,LRR0148,PC-4275,logon,"{""activity"": ""Logon""}"
2,{U6Q3-U0WE70UA-3770UREL},2010-01-02 06:53:04,LRR0148,PC-4124,logon,"{""activity"": ""Logon""}"
3,{D1S0-N6FH62BT-5398KANK},2010-01-02 07:00:00,MOH0273,PC-6699,logon,"{""activity"": ""Logon""}"
4,{I0N5-R7NA26TG-6263KNGM},2010-01-02 07:00:00,IRM0931,PC-7188,logon,"{""activity"": ""Logon""}"
...,...,...,...,...,...,...
1705815,{J8R2-H0FA17MQ-9304VEVQ},2011-05-17 04:40:33,JDF0593,PC-6301,logon,"{""activity"": ""Logoff""}"
1705816,{P3L5-A2AC53JU-4012GIRD},2011-05-17 05:21:58,MLM0950,PC-9787,logon,"{""activity"": ""Logoff""}"
1705817,{P2S0-P1JS62IF-2767BEPH},2011-05-17 05:25:33,IRM0931,PC-7188,logon,"{""activity"": ""Logon""}"
1705818,{Z2U6-I8MA03NC-4963HKBH},2011-05-17 06:23:48,IRM0931,PC-7188,logon,"{""activity"": ""Logoff""}"


In [4]:
def update_logs_df(df: pd.DataFrame) -> pd.DataFrame:
    df['date'] = pd.to_datetime(df['datetime']).dt.date
    df['nday'] = pd.to_datetime(df['date']).dt.day_of_week
    df['is_weekend'] = df['nday'].apply(lambda x: 1 if x in [5, 6] else 0)
    df['date'] = df['date'].astype(str)
    
    return df

In [5]:
def _get_ch_client(host: str, port: int, database: str, username: str, password: str) -> clickhouse_driver.Client:
    ch_client = clickhouse_driver.Client(
        host=host,
        port=port,
        database=database,
        user=username,
        password=password,
        secure=False,
        verify=False,
        compression=False,
        settings={'use_numpy': True},
    )
    return ch_client

In [6]:
def get_df(ch_client: clickhouse_driver.Client, query: str) -> pd.DataFrame:
    rows, columns_defs = ch_client.execute(query=query, with_column_types=True)

    columns = [column_name for column_name, _ in columns_defs]
    df = pd.DataFrame(rows, columns=columns)

    return df

In [7]:
def put_df(ch_client: clickhouse_driver.Client, query: str, df: pd.DataFrame) -> None:
    ch_client.insert_dataframe(
        query=query,
        dataframe=df,
        settings={'use_numpy': True, 'insert_block_size': 50000},
    )

In [26]:
previous_logs = all_raw_logs[(all_raw_logs.datetime >= '2009-02-04') & (all_raw_logs.datetime < '2010-03-01')]

In [27]:
previous_logs = previous_logs[previous_logs['type'] == 'device']
previous_logs = previous_logs[previous_logs.metadata.str.contains('activity": "Connect')].reset_index(drop=True)

In [ ]:
previous_logs['type'].value_counts()

In [ ]:
previous_logs

In [28]:
previous_logs = update_logs_df(previous_logs)
previous_logs

,id,datetime,user,pc,type,metadata,date,nday,is_weekend
0,{J1S3-L9UU75BQ-7790ATPL},2010-01-02 07:21:06,MOH0273,PC-6699,device,"{""activity"": ""Connect""}",2010-01-02,5,1
1,{U1V9-Z7XT67KV-5649MYHI},2010-01-02 07:59:11,HPH0075,PC-2417,device,"{""activity"": ""Connect""}",2010-01-02,5,1
2,{H0Z7-E6GB57XZ-1603MOXD},2010-01-02 07:59:49,IIW0249,PC-0843,device,"{""activity"": ""Connect""}",2010-01-02,5,1
3,{N7C0-Y0QQ16KO-4619HDNN},2010-01-02 08:24:54,HSB0196,PC-8001,device,"{""activity"": ""Connect""}",2010-01-02,5,1
4,{J3A7-Z5JP56DF-7855PJKX},2010-01-02 08:25:18,RRC0553,PC-6672,device,"{""activity"": ""Connect""}",2010-01-02,5,1
...,...,...,...,...,...,...,...,...,...
25654,{E7P7-W1SC93NL-4229UBQB},2010-02-28 16:24:28,HSB0196,PC-8001,device,"{""activity"": ""Connect""}",2010-02-28,6,1
25655,{G9L7-Z7ZF31IS-0312EBIS},2010-02-28 17:06:29,HSB0196,PC-8001,device,"{""activity"": ""Connect""}",2010-02-28,6,1
25656,{H7S6-U9HA19UB-0105DGFY},2010-02-28 17:19:33,HSB0196,PC-8001,device,"{""activity"": ""Connect""}",2010-02-28,6,1
25657,{S7T4-M6VP72IS-8007WICA},2010-02-28 17:54:12,HSB0196,PC-8001,device,"{""activity"": ""Connect""}",2010-02-28,6,1


In [29]:
dl2 = pd.DataFrame({'number_of_logs': previous_logs.groupby(['user','is_weekend','date']).size()}).reset_index()
dl2

,user,is_weekend,date,number_of_logs
0,AAF0535,0,2010-01-05,2
1,AAF0535,0,2010-01-06,1
2,AAF0535,0,2010-01-07,2
3,AAF0535,0,2010-01-08,4
4,AAF0535,0,2010-01-11,1
...,...,...,...,...
7199,ZKS0899,0,2010-02-22,9
7200,ZKS0899,0,2010-02-23,9
7201,ZKS0899,0,2010-02-24,9
7202,ZKS0899,0,2010-02-25,9


In [ ]:
dl3 = pd.DataFrame({'mean': dl2.groupby(['user','is_weekend'])['number_of_logs'].agg('mean')}).reset_index()
dl3

In [ ]:
dl4 = pd.DataFrame({'sum_of_logs': previous_logs.groupby(['user','is_weekend']).size()}).reset_index()
dl4 = dl4[dl4['sum_of_logs'] != 0].reset_index(drop=True)
dl4

In [ ]:
dl4[dl4['user'] == 'ABC0174']

In [ ]:
feature101 = pd.merge(dl2, dl4, on=["user",'is_weekend'])
feature101 = pd.merge(feature101, dl3, on=["user",'is_weekend'])
feature101

In [ ]:
# фича101 с подсчетом среднего отклонения

#feature101['freq'] = 0.0
feature101['freq'] = feature101['number_of_logs'].to_numpy() / feature101['sum_of_logs'].to_numpy()

#feature101['mean_freq'] = 0.0
feature101['mean_freq'] = feature101['mean'].to_numpy() / feature101['sum_of_logs'].to_numpy()

feature101['mean_dev'] = feature101['freq'] - feature101['mean_freq']
feature101['mean_dev'] = feature101['mean_dev'].apply(lambda x: x if x > 0 else 0)

#lst = [0,1] + list(range(2,9))
#feature101 = feature101[feature101.columns[lst]]
feature101

In [17]:
CH_CLIENT = _get_ch_client(
    CH_HOST,
    CH_PORT,
    CH_DATABASE,
    CH_USERNAME,
    CH_PASSWORD
)

In [ ]:
CH_CLIENT.execute(query='select * from nir24.feature_101', with_column_types=True)

In [30]:
dl2['feature'] = 'device_1'
dl2

,user,is_weekend,date,number_of_logs,feature
0,AAF0535,0,2010-01-05,2,device_1
1,AAF0535,0,2010-01-06,1,device_1
2,AAF0535,0,2010-01-07,2,device_1
3,AAF0535,0,2010-01-08,4,device_1
4,AAF0535,0,2010-01-11,1,device_1
...,...,...,...,...,...
7199,ZKS0899,0,2010-02-22,9,device_1
7200,ZKS0899,0,2010-02-23,9,device_1
7201,ZKS0899,0,2010-02-24,9,device_1
7202,ZKS0899,0,2010-02-25,9,device_1


In [31]:
query = 'INSERT INTO logs VALUES'

In [32]:
put_df(CH_CLIENT, query, dl2)

In [ ]:
CH_CLIENT.execute('ALTER TABLE feature_101 DELETE WHERE 1=1')

In [ ]:
features = 'file'

In [40]:
new_logs = all_raw_logs[(all_raw_logs.datetime >= '2010-03-01') & (all_raw_logs.datetime < '2010-03-02')]

In [ ]:
new_logs = df[df['type'] == features]

In [ ]:
df['metadata'][214705]

In [ ]:
new_logs = new_logs[new_logs['type'] == 'logon']
new_logs = new_logs[new_logs.metadata.str.contains('activity": "Logon')].reset_index(drop=True)

In [ ]:
new_logs = update_logs_df(new_logs)
new_logs

In [ ]:
dl2_2 = pd.DataFrame({'number_of_logs': new_logs.groupby(['user','is_weekend','date']).size()}).reset_index()
dl2_2

In [ ]:
feature101

In [ ]:
previous_dl2 = feature101[['user', 'is_weekend', 'date', 'number_of_logs']]
previous_dl2

In [ ]:
all_dl2 = pd.concat([previous_dl2, dl2_2]).reset_index(drop=True)
all_dl2

In [ ]:
all_dl4 = pd.DataFrame({'sum_of_logs': all_dl2.groupby(['user','is_weekend'])['number_of_logs'].sum()}).reset_index()
all_dl4

In [ ]:
all_dl4

In [ ]:
all_dl3 = pd.DataFrame({'mean': all_dl2.groupby(['user','is_weekend'])['number_of_logs'].agg('mean')}).reset_index()
all_dl3

In [ ]:
dl3[dl3['user'] == 'ABC0174']

In [ ]:
all_dl3[all_dl3['user'] == 'ABC0174']

In [56]:

import warnings
from typing import Tuple

import pandas as pd

from app.externals.clickhouse import CH_CLIENT, get_df, insert_df
from app.features.consts import MEAN, SUM

warnings.filterwarnings('ignore')

SELECT_PREVIOUS_LOGS_QUERY = "SELECT user, is_weekend, date, number_of_logs FROM logs where feature = '{}'"
INSERT_LOGS_QUERY = 'INSERT INTO logs VALUES'


def enrich_logs_df(df: pd.DataFrame, features: str) -> pd.DataFrame:
    """

    """
    print(features)
    df = df[df['type'] == features]

    df['date'] = pd.to_datetime(df['datetime']).dt.date
    df['nday'] = pd.to_datetime(df['date']).dt.day_of_week
    df['is_weekend'] = df['nday'].apply(lambda x: 1 if x in [5, 6] else 0)

    df['date'] = df['date'].astype(str)

    return df


def generate_all_logs_df(previous_logs: pd.DataFrame, new_logs: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """

    """
    previous_dl2 = previous_logs[['user', 'is_weekend', 'date', 'number_of_logs']]
    previous_dl2['number_of_logs'] = previous_dl2['number_of_logs'].astype(int)

    dl2 = pd.DataFrame({'number_of_logs': new_logs.groupby(['user', 'is_weekend', 'date']).size()}).reset_index()

    # самое важное - если посчитать all_dl2, то считается всё
    all_dl2 = pd.concat([previous_dl2, dl2]).reset_index(drop=True)

    all_dl3 = pd.DataFrame({MEAN: all_dl2.groupby(['user', 'is_weekend'])['number_of_logs'].agg(MEAN)}).reset_index()
    all_dl4 = pd.DataFrame({SUM: all_dl2.groupby(['user', 'is_weekend'])['number_of_logs'].agg(SUM)}).reset_index()

    feature = pd.merge(all_dl2, all_dl4, on=["user", 'is_weekend'])
    feature = pd.merge(feature, all_dl3, on=["user", 'is_weekend'])

    feature['freq'] = feature['number_of_logs'].to_numpy() / feature[SUM].to_numpy()
    feature['mean_freq'] = feature[MEAN].to_numpy() / feature[SUM].to_numpy()

    feature['mean_dev'] = feature['freq'] - feature['mean_freq']
    feature['mean_dev'] = feature['mean_dev'].apply(lambda x: x if x > 0 else 0)

    # return feature, dl2
    return feature[['user', 'is_weekend', 'date', 'mean_dev']], dl2


def get_previous_logs(features: str) -> pd.DataFrame:
    return get_df(CH_CLIENT, SELECT_PREVIOUS_LOGS_QUERY.format(features))


def send_logs(logs_to_send: pd.DataFrame) -> None:
    insert_df(CH_CLIENT, INSERT_LOGS_QUERY, logs_to_send)


def merge_and_save_features(first_df: pd.DataFrame, second_df: pd.DataFrame=None) -> pd.DataFrame:
    if second_df is None:
        second_df = pd.DataFrame({'user': [], 'is_weekend': [], 'date': []})
    
    merged_df = pd.merge(first_df, second_df, how='outer', on=['user', 'is_weekend', 'date'])
    return merged_df


In [55]:
pd.DataFrame({'user': [], 'is_weekend': [], 'date': []})

,user,is_weekend,date


In [48]:
FEATURES = 'logon'


def logon_1(new_logs: pd.DataFrame) -> pd.DataFrame:
    """
    описание фичи:


    """
    feature_name = 'logon_1'

    # загрузка логов
    previous_logs = get_previous_logs(feature_name)
    new_logs = enrich_logs_df(new_logs, FEATURES)

    # условие
    new_logs = new_logs[new_logs.metadata.str.contains('"activity": "Logon"')].reset_index(drop=True)

    # вычисления и выгрузка логов
    all_logs, new_logs_to_send = generate_all_logs_df(previous_logs, new_logs)
    # send_logs(all_logs)

    return all_logs
    # return new_logs_to_send

In [49]:
FEATURES = 'file'


def file_1(new_logs: pd.DataFrame) -> pd.DataFrame:
    """
    описание фичи:


    """
    feature_name = 'file_1'

    # загрузка логов
    previous_logs = get_previous_logs(feature_name)
    new_logs = enrich_logs_df(new_logs, FEATURES)

    # условие

    # вычисления и выгрузка логов
    all_logs, new_logs_to_send = generate_all_logs_df(previous_logs, new_logs)
    # send_logs(all_logs)

    return all_logs
    # return new_logs_to_send

In [50]:
FEATURES = 'device'


def device_1(new_logs: pd.DataFrame) -> pd.DataFrame:
    """
    описание фичи:


    """
    feature_name = 'device_1'

    # загрузка логов
    previous_logs = get_previous_logs(feature_name)
    new_logs = enrich_logs_df(new_logs, FEATURES)

    # условие
    new_logs = new_logs[new_logs.metadata.str.contains('"activity": "Connect"')].reset_index(drop=True)

    # вычисления и выгрузка логов
    all_logs, new_logs_to_send = generate_all_logs_df(previous_logs, new_logs)
    # send_logs(all_logs)

    return all_logs

In [51]:
USED_FEATURES = [logon_1, device_1, file_1]

In [52]:
#new_logs = pd.DataFrame(instance_data)
for feature in USED_FEATURES:
    df = feature(new_logs)
    print(df.head(10))
    print(df.shape)

device
      user is_weekend        date  mean_dev
0  AAE0190          0  2010-01-04       0.0
1  AAE0190          0  2010-01-05       0.0
2  AAE0190          0  2010-01-06       0.0
3  AAE0190          0  2010-01-07       0.0
4  AAE0190          0  2010-01-08       0.0
5  AAE0190          0  2010-01-11       0.0
6  AAE0190          0  2010-01-12       0.0
7  AAE0190          0  2010-01-13       0.0
8  AAE0190          0  2010-01-14       0.0
9  AAE0190          0  2010-01-15       0.0
(40837, 4)
device
      user is_weekend        date  mean_dev
0  AAF0535          0  2010-01-05  0.000000
1  AAF0535          0  2010-01-06  0.000000
2  AAF0535          0  2010-01-07  0.000000
3  AAF0535          0  2010-01-08  0.027017
4  AAF0535          0  2010-01-11  0.000000
5  AAF0535          0  2010-01-12  0.000000
6  AAF0535          0  2010-01-13  0.000000
7  AAF0535          0  2010-01-14  0.013318
8  AAF0535          0  2010-01-15  0.000000
9  AAF0535          0  2010-01-18  0.000000
(7365, 

In [53]:
df

,user,is_weekend,date,mean_dev
0,AAF0535,0,2010-01-05,0.000000
1,AAF0535,0,2010-01-06,0.033385
2,AAF0535,0,2010-01-07,0.000000
3,AAF0535,0,2010-01-08,0.005216
4,AAF0535,0,2010-01-12,0.000000
...,...,...,...,...
6047,WIF0691,0,2010-03-01,0.000000
6048,WXH0872,0,2010-03-01,0.000000
6049,XHW0498,0,2010-03-01,0.000000
6050,XWD0923,0,2010-03-01,0.000000


In [58]:
merged_df = None
for feature in USED_FEATURES:
    feature = feature(new_logs)
    merged_df = merge_and_save_features(feature, merged_df)

print(merged_df.head(10))
print(merged_df.shape)

device
device
device
      user is_weekend        date  mean_dev  mean_dev_x  mean_dev_y
0  AAF0535          0  2010-01-05  0.000000    0.000000         0.0
1  AAF0535          0  2010-01-06  0.033385    0.000000         0.0
2  AAF0535          0  2010-01-07  0.000000    0.000000         0.0
3  AAF0535          0  2010-01-08  0.005216    0.027017         0.0
4  AAF0535          0  2010-01-12  0.000000    0.000000         0.0
5  AAF0535          0  2010-01-13  0.000000    0.000000         0.0
6  AAF0535          0  2010-01-14  0.000000    0.013318         0.0
7  AAF0535          0  2010-01-15  0.000000    0.000000         0.0
8  AAF0535          0  2010-01-18  0.019301    0.000000         0.0
9  AAF0535          0  2010-01-19  0.000000    0.013318         0.0
(40998, 6)


In [59]:
USED_FEATURES

[<function __main__.logon_1(new_logs: pandas.core.frame.DataFrame) -> pandas.core.frame.DataFrame>,
 <function __main__.device_1(new_logs: pandas.core.frame.DataFrame) -> pandas.core.frame.DataFrame>,
 <function __main__.file_1(new_logs: pandas.core.frame.DataFrame) -> pandas.core.frame.DataFrame>]